- Automated generate test data

- https://docs.ragas.io/en/stable/getstarted/rag_testset_generation

In [14]:
from dotenv import load_dotenv
from ragas.testset import TestsetGenerator
from langchain_community.document_loaders import DirectoryLoader
from ragas.llms.base import llm_factory
from ragas.embeddings import OpenAIEmbeddings
import openai
import os

In [2]:
load_dotenv("configs.env")

True

In [12]:
DATA_DIR_PATH = "data_files"
OUTPUT_TEST_DATA_DIR = "generated_test_data"

In [4]:
doc_files = DirectoryLoader(DATA_DIR_PATH, glob="**/*.txt").load()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


In [5]:
len(doc_files)

2

In [6]:
generator_llm = llm_factory("gpt-4o-mini")
openai_client = openai.OpenAI()
generator_embeddings = OpenAIEmbeddings(
    client=openai_client, model="text-embedding-3-small"
)

In [7]:
generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)

In [8]:
dataset = generator.generate_with_langchain_docs(doc_files, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/1016 [00:00<?, ?it/s]

Applying EmbeddingExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

...

Applying OverlapScoreBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

Skipping multi_hop_abstract_query_synthesizer due to unexpected error: No relationships match the provided condition. Cannot form clusters.


Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/10 [00:00<?, ?it/s]

In [9]:
df = dataset.to_pandas()

In [10]:
df.shape

(10, 4)

In [11]:
df.head()

,user_input,reference_contexts,reference,synthesizer_name
0,Who were the Atkinson brothers?,[The Adventure of the Noble Bachelor XI. The A...,The Atkinson brothers were involved in a singu...,single_hop_specific_query_synthesizer
1,Who is Mary Jane and what is her significance ...,"[You would certainly have been burned, had you...","Mary Jane is described as incorrigible, and he...",single_hop_specific_query_synthesizer
2,What role does Watson play in the interaction ...,[‘Remarkable as being the scene of the death o...,"In the interaction with the Count Von Kramm, W...",single_hop_specific_query_synthesizer
3,How does the context of Bohemia relate to the ...,[to be an immense scandal and seriously compro...,"In the narrative, Bohemia is significant as it...",single_hop_specific_query_synthesizer
4,What details can you provide about the locatio...,[the betrothal was publicly proclaimed. That w...,Briony Lodge is described as a bijou villa loc...,single_hop_specific_query_synthesizer


In [15]:
os.makedirs(OUTPUT_TEST_DATA_DIR, exist_ok=True)

In [16]:
df.to_csv(os.path.join(OUTPUT_TEST_DATA_DIR, "test_data.csv"), index=False)